First we set the SPARQL endpoint and prefixes to get pretty much the same environment as the [Wikidata SPARQL Query Service](https://query.wikidata.org/), the details aren't important

In [1]:
%endpoint https://query.wikidata.org/sparql
%prefix wd: <http://www.wikidata.org/entity/>
%prefix wds: <http://www.wikidata.org/entity/statement/>
%prefix wdv: <http://www.wikidata.org/value/>
%prefix wdt: <http://www.wikidata.org/prop/direct/>
%prefix wikibase: <http://wikiba.se/ontology#>
%prefix p: <http://www.wikidata.org/prop/>
%prefix ps: <http://www.wikidata.org/prop/statement/>
%prefix pq: <http://www.wikidata.org/prop/qualifier/>
%prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
%prefix bd: <http://www.bigdata.com/rdf#>

Endpoint set to: https://query.wikidata.org/sparql
Prefix set: wd: = <http://www.wikidata.org/entity/>
Prefix set: wds: = <http://www.wikidata.org/entity/statement/>
Prefix set: wdv: = <http://www.wikidata.org/value/>
Prefix set: wdt: = <http://www.wikidata.org/prop/direct/>
Prefix set: wikibase: = <http://wikiba.se/ontology#>
Prefix set: p: = <http://www.wikidata.org/prop/>
Prefix set: ps: = <http://www.wikidata.org/prop/statement/>
Prefix set: pq: = <http://www.wikidata.org/prop/qualifier/>
Prefix set: rdfs: = <http://www.w3.org/2000/01/rdf-schema#>
Prefix set: bd: = <http://www.bigdata.com/rdf#>

# Given a gene, get its homologeneID

In [6]:
SELECT ?homologeneID
WHERE
{
    # wd:Q18247422 corresponds to Add1 in house mouse
    wd:Q18247422 wdt:P593 ?homologeneID .
    
    # an entire URI can be used, if it's wrapped in <>
    # <http://www.wikidata.org/entity/Q29723729> wdt:P593 ?homologeneID
}

homologeneID
22758


# Given a homologeneID, get corresponding genes and species

In [3]:
SELECT ?species ?geneLabel ?gene_uri ?tax_uri
WHERE
{
    ?gene_uri wdt:P31 wd:Q7187 ; # ?gene instance of gene
              wdt:P593 "22758" ; # ?gene homologeneID is 22758
              wdt:P703 ?tax_uri . # ?gene is found in NCBI taxonomy id ?tax_id
    
    ?gene_uri rdfs:label ?geneLabel .
    
    ?tax_uri rdfs:label ?species .
    
    FILTER(LANG(?geneLabel) = "en").
    FILTER(LANG(?species) = "en").
}

species,geneLabel,gene_uri,tax_uri
Mus musculus,Add1,http://www.wikidata.org/entity/Q18247422,http://www.wikidata.org/entity/Q83310
Caenorhabditis elegans,add-1,http://www.wikidata.org/entity/Q29674557,http://www.wikidata.org/entity/Q91703
Drosophila melanogaster,hts,http://www.wikidata.org/entity/Q29723729,http://www.wikidata.org/entity/Q130888
Brown Rat,Add1,http://www.wikidata.org/entity/Q24395030,http://www.wikidata.org/entity/Q184224
Homo sapiens,ADD1,http://www.wikidata.org/entity/Q17709877,http://www.wikidata.org/entity/Q15978631


# Get the name and aliases from a gene URI

In [9]:
SELECT ?geneLabel (GROUP_CONCAT(DISTINCT ?geneAltLabel; separator="; ") AS ?geneAltLabel)
WHERE
{
    # https://www.wikidata.org/wiki/Q18247422 is Add1 in mouse
    wd:Q18247422 rdfs:label ?geneLabel ;
                 skos:altLabel ?geneAltLabel .

    FILTER(LANG(?geneLabel) = "en" &&
           LANG(?geneAltLabel) = "en").
}
GROUP BY ?geneLabel

geneLabel,geneAltLabel
Add1,adducin 1 (alpha); AI256389


# Get the species URI given its name

In [10]:
SELECT ?species
WHERE
{
    ?species wdt:P1843 ?taxonCommon.
    FILTER(LANG(?taxonCommon) = "en" &&
           LCASE(STR(?taxonCommon)) = "house mouse").
}

species
http://www.wikidata.org/entity/Q83310
http://www.wikidata.org/entity/Q83310


# Given species name and gene name, find the gene URI

In [16]:
SELECT DISTINCT ?gene
WHERE
{
    ?species wdt:P1843 ?taxonCommon.
    FILTER(LANG(?taxonCommon) = "en" &&
           LCASE(STR(?taxonCommon)) = "house mouse").

    ?gene wdt:P31 wd:Q7187 ;
          rdfs:label ?geneLabel ;
          wdt:P703 ?species .

    FILTER(STR(?geneLabel) = "Add1" &&
           LANG(?geneLabel) = "en").
}

gene
http://www.wikidata.org/entity/Q18247422
